<a href="https://colab.research.google.com/github/simonpwong/data-engineering/blob/master/workshops/workshop1_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [26]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Question 1
limit = 5
generator = square_root_generator(limit)

result = 0
for sqrt_value in generator:
    result += sqrt_value
print(result)


8.382332347441762


In [27]:
# Question 2

limit=13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [8]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [4]:
# Install the dependencies
%%capture
!pip install dlt[duckdb]

In [5]:
# import libraries
import dlt
import duckdb

In [11]:
# Create dlt pipeline, create table for people_1 generator and append people_2 generator to table
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
										table_name="append_generator",
										write_disposition="replace")


print(info)

info = generators_pipeline.run(people_2(),
										table_name="append_generator",
										write_disposition="append")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707500185.3456984 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707500185.9702694 is LOADED and contains no failed jobs


In [28]:
# Question 3
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

appended_generators = conn.sql("SELECT * FROM append_generator ORDER BY name").df()
display(appended_generators)

display(conn.sql("SELECT SUM(age) FROM append_generator"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ append_generator    │
│ generator_1         │
│ merge_generator     │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707500185.3456984,z1hNXFjGRXAkNQ,None
1,2,Person_2,27,City_A,1707500185.3456984,xk8ppZUDBunbjw,None
2,3,Person_3,28,City_A,1707500185.3456984,J/+tMhFWMIi8eA,None
3,3,Person_3,33,City_B,1707500185.9702694,mjRjLBLLe7IlsA,Job_3
4,4,Person_4,29,City_A,1707500185.3456984,NVGSHaX6G1wckA,None
5,4,Person_4,34,City_B,1707500185.9702694,WNOGKQBr1gWISA,Job_4
6,5,Person_5,30,City_A,1707500185.3456984,D69sSG15x3HzxQ,None
7,5,Person_5,35,City_B,1707500185.9702694,XhNQS/mPvHXJ/g,Job_5
8,6,Person_6,36,City_B,1707500185.9702694,aqhDcBuUQD0FWA,Job_6
9,7,Person_7,37,City_B,1707500185.9702694,0kHOXLM3rY1EAw,Job_7


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

In [22]:
# Create new table for people_1 generator and merge with people_2 generator
info = generators_pipeline.run(people_1(),
										table_name="generator_1",
										write_disposition="replace")

print(info)

info = generators_pipeline.run(people_2(),
										table_name="generator_1",
										write_disposition="merge",
                    primary_key="id")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707500416.2987657 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.47 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707500416.9665833 is LOADED and contains no failed jobs


In [29]:
# Question 4
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")


conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

appended_generators = conn.sql("SELECT * FROM generator_1 ORDER BY NAME").df()
display(appended_generators)

display(conn.sql("SELECT sum(age) FROM generator_1"))


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ append_generator    │
│ generator_1         │
│ merge_generator     │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707500416.2987657,6vN5+oXDWLXgRA,None
1,2,Person_2,27,City_A,1707500416.2987657,y3NF+Q9ecg+Vqg,None
2,3,Person_3,33,City_B,1707500416.9665833,HMotn9hIRsT/ZA,Job_3
3,4,Person_4,34,City_B,1707500416.9665833,FFoGW1cqPxrRew,Job_4
4,5,Person_5,35,City_B,1707500416.9665833,Y92r3i8xLcBlJQ,Job_5
5,6,Person_6,36,City_B,1707500416.9665833,xtO7O1h3lXKbdA,Job_6
6,7,Person_7,37,City_B,1707500416.9665833,YSfhj3dnT74QMg,Job_7
7,8,Person_8,38,City_B,1707500416.9665833,8chUbZL0YJOu2g,Job_8


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘